In [1]:
# mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# import module
import sys
sys.path.append('/content/drive/MyDrive')
from preprocessing import FederatedDataBuilder

Mounted at /content/drive


In [6]:
import torch
import torch.nn as nn
import copy
import numpy as np
from torch.utils.data import DataLoader, Subset
from preprocessing import FederatedDataBuilder
from taskarithmetic import SparseSGDM, compute_fisher_sensitivity, calibrate_masks
from fed_avg_iid import DINOCIFAR100, fed_avg_aggregate, GLOBAL_DINO_BACKBONE

# ============================================================
# 1. 本地训练函数 (集成了 Task Arithmetic)
# ============================================================
def local_train_task_arithmetic(model, train_dataset, client_indices, device,
                                 sparsity_ratio=0.1, local_epochs=4):
    """
    客户端本地训练：包含掩码校准和稀疏微调

    关键修复:
    1. 只对head层计算敏感度和应用掩码
    2. backbone保持冻结
    """
    model.train()
    model.to(device)

    # 准备本地数据
    local_sub = Subset(train_dataset, list(client_indices))
    local_loader = DataLoader(local_sub, batch_size=128, shuffle=True)
    criterion = nn.CrossEntropyLoss()

    # --- 阶段 A: 掩码校准 (Mask Calibration) ---
    # 关键修复: 只对head层计算Fisher敏感度
    print(f"  Computing Fisher sensitivity for head parameters only...")

    # 创建一个临时模型，只包含head的参数用于敏感度计算
    head_sensitivity = {}

    # 计算head层的Fisher敏感度
    model.eval()
    for p in model.head.parameters():
        if p.requires_grad:
            head_sensitivity[p] = torch.zeros_like(p.data)

    num_batches = min(len(local_loader), 10)  # 限制batch数量以加速
    processed = 0

    for inputs, labels in local_loader:
        if processed >= num_batches:
            break

        inputs, labels = inputs.to(device), labels.to(device)
        model.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()

        # 只累积head的梯度平方
        for p in model.head.parameters():
            if p.requires_grad and p.grad is not None:
                head_sensitivity[p] += p.grad.data ** 2

        processed += 1

    # 归一化
    for p in head_sensitivity:
        head_sensitivity[p] /= processed

    # 创建掩码
    print(f"  Calibrating masks with sparsity ratio: {sparsity_ratio}")
    masks = calibrate_masks(
        head_sensitivity,
        sparsity_ratio=sparsity_ratio,
        keep_least_sensitive=True
    )

    # 统计实际更新的参数比例
    total_params = sum(p.numel() for p in model.head.parameters())
    masked_params = sum((masks[p] > 0).sum().item() for p in masks)
    actual_sparsity = masked_params / total_params
    print(f"  Actual sparsity: {actual_sparsity:.2%} ({masked_params}/{total_params} params)")

    # --- 阶段 B: 稀疏微调 (Sparse Fine-tuning) ---
    model.train()

    # 只优化head参数
    optimizer = SparseSGDM(
        model.head.parameters(),  # 只优化head
        lr=0.1,  # 提高学习率
        momentum=0.9,
        masks=masks
    )

    epoch_losses = []
    for epoch in range(local_epochs):
        batch_losses = []
        for inputs, labels in local_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()

            # 梯度裁剪
            torch.nn.utils.clip_grad_norm_(model.head.parameters(), max_norm=1.0)

            optimizer.step()
            batch_losses.append(loss.item())

        epoch_loss = np.mean(batch_losses)
        epoch_losses.append(epoch_loss)
        print(f"    Epoch {epoch+1}/{local_epochs}: Loss = {epoch_loss:.4f}")

    return model.state_dict(), len(local_sub)

# ============================================================
# 2. 主联邦训练循环
# ============================================================
def run_fed_iid_task_arithmetic(rounds=50, num_clients=100, sampling_rate=0.1,
                                 sparsity=0.1, local_epochs=4, lr=0.1):
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    print(f"\n{'='*70}")
    print(f"FedAvg + Task Arithmetic (Fixed)")
    print(f"{'='*70}")
    print(f"Clients: {num_clients}, Sampling: {sampling_rate}, Rounds: {rounds}")
    print(f"Sparsity: {sparsity}, Local Epochs: {local_epochs}")
    print(f"Device: {DEVICE}")
    print(f"{'='*70}\n")

    # 数据准备
    print("Preparing data...")
    builder = FederatedDataBuilder(K=num_clients)
    dict_users = builder.get_iid_partition()
    builder.verify_partition(dict_users)

    test_loader = DataLoader(builder.test_dataset, batch_size=256, shuffle=False)

    # 初始化全局模型
    print("\nInitializing global model...")
    global_model = DINOCIFAR100(num_classes=100).to(DEVICE)
    print("✓ Global model ready (using shared DINO backbone)\n")

    history = {"accuracy": [], "loss": [], "round": []}

    for r in range(rounds):
        print(f"\n{'='*70}")
        print(f"Round {r+1}/{rounds} (Sparsity: {sparsity})")
        print(f"{'='*70}")

        local_weights = []
        local_counts = []

        # 随机选择客户端
        m = max(int(sampling_rate * num_clients), 1)
        selected_clients = np.random.choice(range(num_clients), m, replace=False)
        print(f"Selected {m} clients: {selected_clients[:5]}..." if m > 5 else f"Selected clients: {selected_clients}")

        for idx, client_id in enumerate(selected_clients):
            print(f"\nClient {idx+1}/{m} (ID: {client_id}):")

            # 深拷贝全局模型到本地
            local_model = copy.deepcopy(global_model)

            # 本地任务算术训练
            w, count = local_train_task_arithmetic(
                local_model,
                builder.train_dataset,
                dict_users[client_id],
                DEVICE,
                sparsity_ratio=sparsity,
                local_epochs=local_epochs
            )

            local_weights.append(w)
            local_counts.append(count)

        # 聚合 (FedAvg)
        print(f"\nAggregating {len(local_weights)} local models...")
        global_weights = fed_avg_aggregate(global_model, local_weights, local_counts)
        global_model.load_state_dict(global_weights, strict=False)

        # 全局评估
        test_loss, test_acc = evaluate(global_model, test_loader, DEVICE)
        history["accuracy"].append(test_acc)
        history["loss"].append(test_loss)
        history["round"].append(r + 1)

        print(f"\n{'='*70}")
        print(f"Round {r+1} Results:")
        print(f"  Test Loss: {test_loss:.4f}")
        print(f"  Test Accuracy: {test_acc:.2f}%")
        print(f"{'='*70}")

    # 最终结果
    print(f"\n{'='*70}")
    print(f"Training Complete!")
    print(f"{'='*70}")
    print(f"Final Test Accuracy: {history['accuracy'][-1]:.2f}%")
    print(f"Best Test Accuracy: {max(history['accuracy']):.2f}%")
    print(f"{'='*70}\n")

    return history

def evaluate(model, loader, device):
    model.eval()
    criterion = nn.CrossEntropyLoss()
    correct = 0
    total = 0
    total_loss = 0

    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            total_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    return total_loss / len(loader), 100 * correct / total

if __name__ == "__main__":
    # 测试不同稀疏度
    for sparsity in [0.1, 0.3, 0.5]:
        print(f"\n\n{'#'*70}")
        print(f"# Testing with sparsity = {sparsity}")
        print(f"{'#'*70}\n")

        history = run_fed_iid_task_arithmetic(
            rounds=30,
            num_clients=100,
            sampling_rate=0.1,
            sparsity=sparsity,
            local_epochs=4
        )

Creating IID partition for 100 clients...
Downloading/Loading DINO ViT-S/16...


Using cache found in /root/.cache/torch/hub/facebookresearch_dino_main



--- Round 1/20 (Sparsity: 0.5) ---
Calculating sensitivity over 5 batches...
Calculating sensitivity over 5 batches...
Calculating sensitivity over 5 batches...
Calculating sensitivity over 5 batches...
Calculating sensitivity over 5 batches...
Calculating sensitivity over 5 batches...
Calculating sensitivity over 5 batches...
Calculating sensitivity over 5 batches...
Calculating sensitivity over 5 batches...
Calculating sensitivity over 5 batches...
Calculating sensitivity over 5 batches...
Calculating sensitivity over 5 batches...
Calculating sensitivity over 5 batches...
Calculating sensitivity over 5 batches...
Calculating sensitivity over 5 batches...
Calculating sensitivity over 5 batches...
Calculating sensitivity over 5 batches...
Calculating sensitivity over 5 batches...
Calculating sensitivity over 5 batches...
Calculating sensitivity over 5 batches...
Calculating sensitivity over 5 batches...
Calculating sensitivity over 5 batches...
Calculating sensitivity over 5 batches..

KeyboardInterrupt: 

In [ ]:
"""
联邦Task Arithmetic诊断脚本 (最终版)
用于排查为什么模型精度极低(~1%)

使用 DINOCIFAR100 模型类
"""

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Subset
import numpy as np

from preprocessing import FederatedDataBuilder
from taskarithmetic import compute_fisher_sensitivity, calibrate_masks

# 导入模型 - 兼容不同的命名
try:
    from fed_avg_iid import DINOCIFAR100Fixed as DINOCIFAR100
except ImportError:
    from fed_avg_iid import DINOCIFAR100


def diagnose_mask_problem(sparsity_ratio=0.1):
    """
    诊断掩码是否过于严格
    """
    print("\n" + "="*70)
    print("诊断 1: 检查掩码生成")
    print("="*70)

    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # 准备数据
    builder = FederatedDataBuilder(K=10)
    dict_users = builder.get_iid_partition()

    # 创建模型
    model = DINOCIFAR100(num_classes=100).to(DEVICE)

    # 准备一个客户端的数据
    local_subset = Subset(builder.train_dataset, list(dict_users[0]))
    local_loader = DataLoader(local_subset, batch_size=32, shuffle=True)

    criterion = nn.CrossEntropyLoss()

    # 计算敏感度
    print(f"\n计算Fisher敏感度 (sparsity={sparsity_ratio})...")
    sensitivity_scores = compute_fisher_sensitivity(
        model, local_loader, criterion, DEVICE, num_batches=5
    )

    # 生成掩码
    masks = calibrate_masks(
        sensitivity_scores,
        sparsity_ratio=sparsity_ratio,
        keep_least_sensitive=True
    )

    # 分析掩码
    print("\n掩码统计:")
    print("-" * 70)

    total_params = 0
    frozen_params = 0
    active_params = 0

    for name, param in model.named_parameters():
        if param.requires_grad:
            mask = masks.get(param)
            if mask is not None:
                num_params = int(param.numel())
                num_active = int(mask.sum().item())
                num_frozen = num_params - num_active

                total_params += num_params
                frozen_params += num_frozen
                active_params += num_active

                active_ratio = 100 * num_active / num_params
                print(f"{name:30s} | Total: {num_params:8d} | "
                      f"Active: {num_active:8d} ({active_ratio:5.1f}%) | "
                      f"Frozen: {num_frozen:8d}")

    print("-" * 70)
    print(f"{'总计':30s} | Total: {total_params:8d} | "
          f"Active: {active_params:8d} ({100*active_params/total_params:5.1f}%) | "
          f"Frozen: {frozen_params:8d}")

    # 关键检查
    if active_params == 0:
        print("\n❌ 严重错误: 所有参数都被冻结!")
        print("   - 模型无法学习")
        print("   - 需要检查calibrate_masks实现")
        return False

    if active_params < total_params * 0.01:  # 小于1%
        print("\n⚠️  警告: 可更新参数过少!")
        print(f"   - 只有{100*active_params/total_params:.2f}%的参数可以更新")
        print("   - 建议增大sparsity_ratio")
        return False

    print("\n✓ 掩码生成正常")
    return True


def diagnose_training_step():
    """
    诊断单步训练是否正常
    """
    print("\n" + "="*70)
    print("诊断 2: 检查训练步骤")
    print("="*70)

    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # 准备数据
    builder = FederatedDataBuilder(K=10)
    dict_users = builder.get_iid_partition()

    # 创建模型
    model = DINOCIFAR100(num_classes=100).to(DEVICE)

    # 检查backbone是否冻结
    print("\n检查backbone冻结状态:")
    backbone_params_trainable = sum(p.requires_grad for p in model.backbone.parameters())
    print(f"Backbone可训练参数数: {backbone_params_trainable}")
    if backbone_params_trainable > 0:
        print("❌ 错误: Backbone应该被完全冻结!")
        return False
    print("✓ Backbone已正确冻结")

    # 检查head
    print("\nHead参数:")
    for name, param in model.head.named_parameters():
        print(f"  {name}: requires_grad={param.requires_grad}, shape={param.shape}")

    # 准备本地数据
    local_subset = Subset(builder.train_dataset, list(dict_users[0]))
    local_loader = DataLoader(local_subset, batch_size=32, shuffle=True)

    # 获取一个batch
    inputs, targets = next(iter(local_loader))
    inputs, targets = inputs.to(DEVICE), targets.to(DEVICE)

    # 前向传播
    print("\n测试前向传播:")
    model.eval()
    with torch.no_grad():
        outputs = model(inputs)
        print(f"  输出形状: {outputs.shape}")
        print(f"  输出范围: [{outputs.min().item():.2f}, {outputs.max().item():.2f}]")

        # 检查初始精度
        _, predicted = outputs.max(1)
        correct = predicted.eq(targets).sum().item()
        acc = 100. * correct / targets.size(0)
        print(f"  初始精度 (随机): {acc:.2f}%")

        if acc < 0.5 or acc > 5:
            print(f"  ⚠️  警告: 初始精度异常 (期望~1%)")

    # 测试反向传播
    print("\n测试反向传播:")
    model.train()
    criterion = nn.CrossEntropyLoss()

    # 记录初始权重
    initial_weight = model.head.weight.clone()

    # 训练一步
    optimizer = torch.optim.SGD(model.head.parameters(), lr=0.1)
    optimizer.zero_grad()
    outputs = model(inputs)
    loss = criterion(outputs, targets)
    loss.backward()

    # 检查梯度
    if model.head.weight.grad is None:
        print("  ❌ 错误: 没有计算梯度!")
        return False

    grad_norm = model.head.weight.grad.norm().item()
    print(f"  梯度范数: {grad_norm:.4f}")

    if grad_norm < 1e-6:
        print("  ⚠️  警告: 梯度过小")

    # 更新权重
    optimizer.step()

    # 检查权重是否改变
    weight_change = (model.head.weight - initial_weight).abs().max().item()
    print(f"  权重最大变化: {weight_change:.6f}")

    if weight_change < 1e-8:
        print("  ❌ 错误: 权重没有更新!")
        return False

    print("  ✓ 训练步骤正常")
    return True


def diagnose_aggregation():
    """
    诊断聚合是否正常
    """
    print("\n" + "="*70)
    print("诊断 3: 检查FedAvg聚合")
    print("="*70)

    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    from fed_avg_iid import fed_avg_aggregate

    # 创建全局模型
    global_model = DINOCIFAR100(num_classes=100).to(DEVICE)

    # 创建两个模拟的本地模型权重
    local_weights = []

    for i in range(2):
        local_model = DINOCIFAR100(num_classes=100).to(DEVICE)
        # 随机修改权重
        with torch.no_grad():
            local_model.head.weight += torch.randn_like(local_model.head.weight) * 0.1
        local_weights.append(local_model.state_dict())

    client_counts = [100, 100]

    # 执行聚合
    print("\n执行聚合...")
    global_weight_before = global_model.head.weight.clone()

    new_weights = fed_avg_aggregate(global_model, local_weights, client_counts)
    global_model.load_state_dict(new_weights, strict=False)

    global_weight_after = global_model.head.weight

    # 检查权重是否改变
    weight_change = (global_weight_after - global_weight_before).abs().max().item()
    print(f"全局模型权重最大变化: {weight_change:.6f}")

    if weight_change < 1e-8:
        print("❌ 错误: 聚合后全局模型权重没有改变!")
        return False

    print("✓ 聚合正常")
    return True


def test_without_task_arithmetic():
    """
    测试不使用Task Arithmetic的标准FedAvg
    """
    print("\n" + "="*70)
    print("诊断 4: 测试标准FedAvg (无Task Arithmetic)")
    print("="*70)

    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # 数据准备
    builder = FederatedDataBuilder(K=10)
    dict_users = builder.get_iid_partition()

    test_loader = DataLoader(
        builder.test_dataset,
        batch_size=256,
        shuffle=False
    )

    # 全局模型
    global_model = DINOCIFAR100(num_classes=100).to(DEVICE)

    from fed_avg_iid import fed_avg_aggregate, evaluate_global

    print("\n运行3轮标准FedAvg...")

    for r in range(3):
        # 选择2个客户端
        selected_clients = np.random.choice(range(10), 2, replace=False)

        local_weights = []
        client_counts = []

        for client_idx in selected_clients:
            # 本地训练
            local_model = DINOCIFAR100(num_classes=100).to(DEVICE)
            local_model.load_state_dict(global_model.state_dict())

            local_subset = Subset(builder.train_dataset, list(dict_users[client_idx]))
            local_loader = DataLoader(local_subset, batch_size=32, shuffle=True)

            optimizer = torch.optim.SGD(local_model.head.parameters(), lr=0.1, momentum=0.9)
            criterion = nn.CrossEntropyLoss()

            local_model.train()
            step_count = 0
            iterator = iter(local_loader)

            # 正确实现J=4步
            while step_count < 4:
                try:
                    inputs, targets = next(iterator)
                    inputs, targets = inputs.to(DEVICE), targets.to(DEVICE)

                    optimizer.zero_grad()
                    outputs = local_model(inputs)
                    loss = criterion(outputs, targets)
                    loss.backward()
                    optimizer.step()

                    step_count += 1
                except StopIteration:
                    break

            local_weights.append(local_model.state_dict())
            client_counts.append(len(dict_users[client_idx]))

        # 聚合
        new_weights = fed_avg_aggregate(global_model, local_weights, client_counts)
        global_model.load_state_dict(new_weights, strict=False)

        # 评估
        test_loss, test_acc = evaluate_global(global_model, test_loader, DEVICE)
        print(f"Round {r+1}: Test Acc = {test_acc:.2f}%")

        if test_acc < 1.0:
            print("  ⚠️  精度仍然过低!")
        elif test_acc > 3.0:
            print("  ✓ 精度开始提升,基础流程正常")
            return True

    return False


def main():
    """
    运行所有诊断
    """
    print("\n" + "🔍"*35)
    print("      联邦Task Arithmetic 诊断工具")
    print("🔍"*35)

    # 诊断1: 掩码
    mask_ok = diagnose_mask_problem(sparsity_ratio=0.1)

    # 诊断2: 训练步骤
    training_ok = diagnose_training_step()

    # 诊断3: 聚合
    aggregation_ok = diagnose_aggregation()

    # 诊断4: 无TA的FedAvg
    fedavg_ok = test_without_task_arithmetic()

    # 总结
    print("\n" + "="*70)
    print("诊断总结")
    print("="*70)
    print(f"1. 掩码生成: {'✓ 正常' if mask_ok else '❌ 异常'}")
    print(f"2. 训练步骤: {'✓ 正常' if training_ok else '❌ 异常'}")
    print(f"3. FedAvg聚合: {'✓ 正常' if aggregation_ok else '❌ 异常'}")
    print(f"4. 标准FedAvg: {'✓ 正常' if fedavg_ok else '❌ 异常'}")

    print("\n" + "="*70)
    print("建议:")
    print("="*70)

    if not mask_ok:
        print("1. 检查calibrate_masks函数实现")
        print("2. 尝试更大的sparsity_ratio (如0.5)")
        print("3. 确认keep_least_sensitive逻辑正确")

    if not training_ok:
        print("1. 检查模型初始化")
        print("2. 确认backbone正确冻结")
        print("3. 调整学习率")

    if not fedavg_ok:
        print("1. 基础FedAvg就有问题,先修复它")
        print("2. 检查数据加载")
        print("3. 增加本地训练步数")

    if mask_ok and training_ok and aggregation_ok and not fedavg_ok:
        print("1. 问题可能在数据处理或模型架构")
        print("2. 尝试运行fed_avg_iid.py看是否正常")

    print("\n💡 快速修复建议:")
    print("   - 先确保标准FedAvg能work (精度>5%)")
    print("   - 再加入Task Arithmetic")
    print("   - 使用较大的sparsity_ratio开始测试")


if __name__ == "__main__":
    main()


🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍
      联邦Task Arithmetic 诊断工具
🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍

诊断 1: 检查掩码生成
Creating IID partition for 10 clients...
Downloading/Loading DINO ViT-S/16...


Using cache found in /root/.cache/torch/hub/facebookresearch_dino_main



计算Fisher敏感度 (sparsity=0.1)...
Calculating sensitivity over 5 batches...

掩码统计:
----------------------------------------------------------------------
backbone.cls_token             | Total:      384 | Active:        0 (  0.0%) | Frozen:      384
backbone.pos_embed             | Total:    75648 | Active:      316 (  0.4%) | Frozen:    75332
backbone.patch_embed.proj.weight | Total:   294912 | Active:        0 (  0.0%) | Frozen:   294912
backbone.patch_embed.proj.bias | Total:      384 | Active:        0 (  0.0%) | Frozen:      384
backbone.blocks.0.norm1.weight | Total:      384 | Active:      207 ( 53.9%) | Frozen:      177
backbone.blocks.0.norm1.bias   | Total:      384 | Active:       44 ( 11.5%) | Frozen:      340
backbone.blocks.0.attn.qkv.weight | Total:   442368 | Active:   351693 ( 79.5%) | Frozen:    90675
backbone.blocks.0.attn.qkv.bias | Total:     1152 | Active:      685 ( 59.5%) | Frozen:      467
backbone.blocks.0.attn.proj.weight | Total:   147456 | Active:    50960 ( 3

Using cache found in /root/.cache/torch/hub/facebookresearch_dino_main



检查backbone冻结状态:
Backbone可训练参数数: 150
❌ 错误: Backbone应该被完全冻结!

诊断 3: 检查FedAvg聚合
Downloading/Loading DINO ViT-S/16...


...


运行3轮标准FedAvg...
Downloading/Loading DINO ViT-S/16...


Using cache found in /root/.cache/torch/hub/facebookresearch_dino_main


Downloading/Loading DINO ViT-S/16...


Using cache found in /root/.cache/torch/hub/facebookresearch_dino_main


Round 1: Test Acc = 6.55%
  ✓ 精度开始提升,基础流程正常

诊断总结
1. 掩码生成: ✓ 正常
2. 训练步骤: ❌ 异常
3. FedAvg聚合: ✓ 正常
4. 标准FedAvg: ✓ 正常

建议:
1. 检查模型初始化
2. 确认backbone正确冻结
3. 调整学习率

💡 快速修复建议:
   - 先确保标准FedAvg能work (精度>5%)
   - 再加入Task Arithmetic
   - 使用较大的sparsity_ratio开始测试
